In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
os.chdir("/Users/bjarnihaukurbjarnason/Documents/genki-signals")

import numpy as np

In [5]:
from genki_signals.signal_system import SignalSystem
from genki_signals.signal_sources import WaveSignalSource
import genki_signals.signal_functions as sf

wave_source = WaveSignalSource(
    ble_address = "D6FC2BBF-EADE-F52E-DEEA-080D63FC2AF7",
    spectrogram = True
)

derived = [
    sf.Stack("acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z", name="model_input"),
    sf.Inference(
        "model_input",
        name="model_output",
        model_filename="genki_signals/models/is_touching_model/model.onnx",
        stateful=True,
        init_state=np.ones((1, 512), dtype=np.float32)
    )
]

system = SignalSystem(wave_source, derived)

In [6]:
system.start()

Connecting to wave at address D6FC2BBF-EADE-F52E-DEEA-080D63FC2AF7
Connected to Wave


In [7]:
from genki_signals.buffers import DataBuffer

buffer = DataBuffer(maxlen=400)

system.register_data_feed(id(buffer), lambda d: buffer.extend(d))

In [8]:
buffer

DataBuffer(max_size=400, data=acc_x: (16, 39)
acc_y: (16, 39)
acc_z: (16, 39)
gyro_x: (16, 39)
gyro_y: (16, 39)
gyro_z: (16, 39)
timestamp_us: (39,)
model_input: (6, 16, 39)
model_output: (4, 28))

In [9]:
from genki_signals.signal_frontends import WidgetFrontend, Line, Bar

bar = Bar(
    "model_output",
    y_range=(0,1),
    x_names=["touching_static", "touching_moving", "air_static", "air_moving"]
)

frontend = WidgetFrontend(system, [bar])

In [10]:
bar.widget

Figure(axes=[Axis(label='indices', scale=OrdinalScale(domain=['touching_static', 'touching_moving', 'air_stati…

In [11]:
from genki_widgets import Spectrogram

spectrogram = Spectrogram(
    ("acc_y", -1),
    window_size=32,
    sample_rate=2000,
    n_visible_windows=500,
    colormap_max = 0.0005
)

frontend.register_update_callback(id(spectrogram), lambda d: spectrogram.update(d.as_dict()))

In [12]:
spectrogram

Spectrogram(colormap_max=0.0005, data={'acc_x': [[7.459411335730692e-06, 0.00010834930435521528], [3.217010453…

In [16]:
frontend.deregister_update_callback(id(spectrogram))